# 代码测试
Pulp包需要第三方下载
-pip install pulp

In [2]:
#算法部分，使用pulp内置cdc求解器
import pulp
import numpy as np
import pandas as pd
class ILP():
    def __init__(self,c,T,S):
        #输入说明
        #T:饲料厂产量
        #S:养殖场需求
        #l：距离矩阵
        self.T = T
        self.S = S
        self.row = len(self.T)
        self.col = len(self.S)
        self.c = np.array(c).reshape(self.row,self.col)
        
    def solve(self):
        row = len(self.T) #饲料厂个数
        col = len(self.S) #养殖场个数
        
        prob = pulp.LpProblem('运输配给优化', sense=pulp.LpMinimize)
        
        var = [[pulp.LpVariable(f'x{i}_{j}', cat=pulp.LpBinary) for j in range(col)] for i in range(row)]

        flatten = lambda x: [y for l in x for y in flatten(l)] if type(x) is list else [x]

        prob += pulp.lpDot(flatten(var), self.c.flatten())

        for i in range(row):
            prob += (pulp.lpDot(pulp.lpSum(var[i]),self.S) <= self.T[i])

        for j in range(col):
            prob += (pulp.lpDot([var[i][j] for i in range(row)], [1]*row) == 1)
        pulp.LpSolverDefault.msg = 1
        prob.solve()
        self.opt_val = round(pulp.value(prob.objective),2)
        self.opt_x = pd.DataFrame([[pulp.value(var[i][j]) for j in range(col)] for i in range(row)])
        #输出说明：
        #opt_val为最低运输距离
        #opt_x为最优运输方式，格式为row（饲料厂数）*col（养殖场数）的0-1矩阵。x[i,j]代表第i间饲料厂是否给第j间养殖场配送饲料
        
#部分数据测试及结果

# 算法测试部分

In [3]:
#加载使用的package以及读取相关文件
#测试2020年1月份数据
import pulp
import numpy as np
import pandas as pd
月份 = '2020-01'
距离矩阵 = pd.read_excel('Data1.xlsx')
养殖场 = pd.read_csv('Data2.csv',encoding = 'gbk')
饲料厂 = pd.read_excel('Data3.xlsx')
距离矩阵 = 距离矩阵[['FieldName','FeedName','FDistance']]
距离矩阵 = 距离矩阵.rename(columns = {'FieldName':'养殖场','FeedName' : '饲料厂', 'FDistance' : '距离'})
养殖场 = 养殖场[养殖场.月份 == 月份][['场区名称','日需求量']]
养殖场 = 养殖场.rename(columns = {'场区名称':'养殖场'})
饲料厂 =饲料厂[['FProjectName','FScaleAmount']]
饲料厂 = 饲料厂.rename(columns = {'FProjectName':'饲料厂','FScaleAmount':'产能'}).dropna()

In [4]:
#查看缺失值
养殖场.日需求量.isnull().sum()
饲料厂.产能.isnull().sum()

0

In [5]:
#获得用来制作矩阵的pivot table
data = pd.merge(距离矩阵,养殖场,how = 'inner',on = '养殖场')
data = pd.merge(data,饲料厂,how = 'inner', on = '饲料厂')
data

,养殖场,饲料厂,距离,日需求量,产能
0,唐河1场,钟祥饲料厂1厂,287484,24.51,36.0
1,唐河2场,钟祥饲料厂1厂,358370,206.82,36.0
2,唐河3场,钟祥饲料厂1厂,357624,30.88,36.0
3,唐河4场,钟祥饲料厂1厂,305330,45.74,36.0
4,唐河5场,钟祥饲料厂1厂,293453,41.53,36.0
5,唐河7场,钟祥饲料厂1厂,344204,49.95,36.0
6,卧龙1场,钟祥饲料厂1厂,345229,139.89,36.0
7,卧龙5场,钟祥饲料厂1厂,301336,17.48,36.0
8,卧龙7场,钟祥饲料厂1厂,294783,60.13,36.0
9,唐河10场,钟祥饲料厂1厂,347563,12.14,36.0


In [6]:
df1 = pd.pivot_table(data,index=['养殖场'],columns = ['饲料厂'],values = '距离')
print(df1.index[np.where(np.isnan(df1))[0]], df1.columns[np.where(np.isnan(df1))[1]])
#缺失颍上2场到万荣饲料厂1厂 的距离。这个数字在测试阶段我选择随机填写为500公里
#在这里也可以选择删掉所有和'颍上2场'或与'万荣饲料厂1厂'有关的数据
#如果爬虫获得的数据完整这里应该没有缺失值
df1

Index(['颍上2场'], dtype='object', name='养殖场') Index(['万荣饲料厂1厂'], dtype='object', name='饲料厂')


饲料厂,万荣饲料厂1厂,万荣饲料机组1场,上蔡饲料1厂,东明饲料厂1厂,东明饲料机组1厂,义县饲料1厂,克东饲料1厂,兰西饲料机组1厂,内乡饲料1厂,内乡饲料2厂,...,阜新饲料厂1厂,阜新饲料机组1厂,陈仓饲料厂1厂,颍上饲料厂,颍泉饲料1厂,馆陶饲料1厂,鹿邑饲料厂,黑山饲料1厂,龙江4场饲料机组,龙江饲料1厂
养殖场,,,,,,,,,,,,,,,,,,,,,
万荣1场,9962.0,12555.0,530945.0,467244.0,465389.0,1369274.0,2306783.0,2115801.0,361015.0,371934.0,...,1498156.0,1507273.0,436337.0,721824.0,647211.0,524661.0,540855.0,1457483.0,2019357.0,2019356.0
万荣2场,20378.0,23543.0,561813.0,484975.0,483120.0,1372157.0,2309666.0,2118684.0,368285.0,379204.0,...,1501039.0,1510156.0,429311.0,752871.0,678258.0,527544.0,575810.0,1460366.0,2022240.0,2022239.0
万荣3场,8088.0,11253.0,536386.0,472685.0,470830.0,1368488.0,2305997.0,2115015.0,372315.0,383234.0,...,1497370.0,1506487.0,445928.0,727265.0,652652.0,523875.0,546296.0,1456697.0,2018571.0,2018570.0
万荣4场,6035.0,9200.0,534333.0,470632.0,468777.0,1366435.0,2303944.0,2112962.0,370262.0,381181.0,...,1495317.0,1504434.0,443875.0,725212.0,650599.0,521822.0,544243.0,1454644.0,2016518.0,2016517.0
万荣5场,44793.0,47386.0,512392.0,461058.0,459203.0,1363023.0,2300532.0,2109550.0,355880.0,366799.0,...,1491905.0,1501022.0,431202.0,703450.0,628837.0,518410.0,534669.0,1451232.0,2013106.0,2013105.0
上蔡2场,497092.0,499685.0,61549.0,271069.0,267464.0,1347648.0,2294351.0,2094175.0,250701.0,260030.0,...,1476530.0,1485647.0,770800.0,242932.0,209964.0,440975.0,140045.0,1435857.0,2082616.0,2082615.0
上蔡3场,509294.0,511887.0,73751.0,283271.0,279666.0,1359850.0,2306553.0,2106377.0,262903.0,272232.0,...,1488732.0,1497849.0,783002.0,255134.0,222166.0,453177.0,152247.0,1448059.0,2094818.0,2094817.0
上蔡5场,514919.0,517512.0,76772.0,249056.0,245451.0,1325635.0,2272338.0,2072162.0,265924.0,275253.0,...,1454517.0,1463634.0,786023.0,262564.0,187951.0,418962.0,118032.0,1413844.0,2060603.0,2060602.0
东明14场,461190.0,463783.0,301708.0,13292.0,22542.0,1095049.0,2041752.0,1841576.0,422697.0,432026.0,...,1223931.0,1233048.0,778119.0,383227.0,333653.0,213881.0,217621.0,1183258.0,1894205.0,1894204.0


如果距离不完整，算法程序会报错。
这里因为缺少了颍上2场到万荣饲料厂1厂的距离，制作的pivot table里关于颍上2场的日需求量需要手动填写

In [7]:
df2 = pd.pivot_table(data,index=['养殖场'],columns = ['饲料厂'],values = '日需求量')
print(df2.index[np.where(np.isnan(df2))[0]])
df2

Index(['颍上2场'], dtype='object', name='养殖场')


饲料厂,万荣饲料厂1厂,万荣饲料机组1场,上蔡饲料1厂,东明饲料厂1厂,东明饲料机组1厂,义县饲料1厂,克东饲料1厂,兰西饲料机组1厂,内乡饲料1厂,内乡饲料2厂,...,阜新饲料厂1厂,阜新饲料机组1厂,陈仓饲料厂1厂,颍上饲料厂,颍泉饲料1厂,馆陶饲料1厂,鹿邑饲料厂,黑山饲料1厂,龙江4场饲料机组,龙江饲料1厂
养殖场,,,,,,,,,,,,,,,,,,,,,
万荣1场,18.37,18.37,18.37,18.37,18.37,18.37,18.37,18.37,18.37,18.37,...,18.37,18.37,18.37,18.37,18.37,18.37,18.37,18.37,18.37,18.37
万荣2场,38.63,38.63,38.63,38.63,38.63,38.63,38.63,38.63,38.63,38.63,...,38.63,38.63,38.63,38.63,38.63,38.63,38.63,38.63,38.63,38.63
万荣3场,17.92,17.92,17.92,17.92,17.92,17.92,17.92,17.92,17.92,17.92,...,17.92,17.92,17.92,17.92,17.92,17.92,17.92,17.92,17.92,17.92
万荣4场,65.98,65.98,65.98,65.98,65.98,65.98,65.98,65.98,65.98,65.98,...,65.98,65.98,65.98,65.98,65.98,65.98,65.98,65.98,65.98,65.98
万荣5场,33.66,33.66,33.66,33.66,33.66,33.66,33.66,33.66,33.66,33.66,...,33.66,33.66,33.66,33.66,33.66,33.66,33.66,33.66,33.66,33.66
上蔡2场,17.76,17.76,17.76,17.76,17.76,17.76,17.76,17.76,17.76,17.76,...,17.76,17.76,17.76,17.76,17.76,17.76,17.76,17.76,17.76,17.76
上蔡3场,32.15,32.15,32.15,32.15,32.15,32.15,32.15,32.15,32.15,32.15,...,32.15,32.15,32.15,32.15,32.15,32.15,32.15,32.15,32.15,32.15
上蔡5场,18.78,18.78,18.78,18.78,18.78,18.78,18.78,18.78,18.78,18.78,...,18.78,18.78,18.78,18.78,18.78,18.78,18.78,18.78,18.78,18.78
东明14场,25.91,25.91,25.91,25.91,25.91,25.91,25.91,25.91,25.91,25.91,...,25.91,25.91,25.91,25.91,25.91,25.91,25.91,25.91,25.91,25.91


In [8]:
df3 = pd.pivot_table(data,index=['养殖场'],columns = ['饲料厂'],values = '产能')
print(df3.columns[np.where(np.isnan(df3))[1]])
df3

Index(['万荣饲料厂1厂'], dtype='object', name='饲料厂')


饲料厂,万荣饲料厂1厂,万荣饲料机组1场,上蔡饲料1厂,东明饲料厂1厂,东明饲料机组1厂,义县饲料1厂,克东饲料1厂,兰西饲料机组1厂,内乡饲料1厂,内乡饲料2厂,...,阜新饲料厂1厂,阜新饲料机组1厂,陈仓饲料厂1厂,颍上饲料厂,颍泉饲料1厂,馆陶饲料1厂,鹿邑饲料厂,黑山饲料1厂,龙江4场饲料机组,龙江饲料1厂
养殖场,,,,,,,,,,,,,,,,,,,,,
万荣1场,30.0,9.0,30.0,30.0,9.0,24.0,30.0,9.0,48.0,36.0,...,30.0,9.0,30.0,30.0,18.0,24.0,30.0,24.0,9.0,30.0
万荣2场,30.0,9.0,30.0,30.0,9.0,24.0,30.0,9.0,48.0,36.0,...,30.0,9.0,30.0,30.0,18.0,24.0,30.0,24.0,9.0,30.0
万荣3场,30.0,9.0,30.0,30.0,9.0,24.0,30.0,9.0,48.0,36.0,...,30.0,9.0,30.0,30.0,18.0,24.0,30.0,24.0,9.0,30.0
万荣4场,30.0,9.0,30.0,30.0,9.0,24.0,30.0,9.0,48.0,36.0,...,30.0,9.0,30.0,30.0,18.0,24.0,30.0,24.0,9.0,30.0
万荣5场,30.0,9.0,30.0,30.0,9.0,24.0,30.0,9.0,48.0,36.0,...,30.0,9.0,30.0,30.0,18.0,24.0,30.0,24.0,9.0,30.0
上蔡2场,30.0,9.0,30.0,30.0,9.0,24.0,30.0,9.0,48.0,36.0,...,30.0,9.0,30.0,30.0,18.0,24.0,30.0,24.0,9.0,30.0
上蔡3场,30.0,9.0,30.0,30.0,9.0,24.0,30.0,9.0,48.0,36.0,...,30.0,9.0,30.0,30.0,18.0,24.0,30.0,24.0,9.0,30.0
上蔡5场,30.0,9.0,30.0,30.0,9.0,24.0,30.0,9.0,48.0,36.0,...,30.0,9.0,30.0,30.0,18.0,24.0,30.0,24.0,9.0,30.0
东明14场,30.0,9.0,30.0,30.0,9.0,24.0,30.0,9.0,48.0,36.0,...,30.0,9.0,30.0,30.0,18.0,24.0,30.0,24.0,9.0,30.0


In [9]:
#获得距离矩阵，填补缺失值
l = np.array(df1)
S = np.array(df2.iloc[:,0])
T = np.array(df3.iloc[0,:])*10000/330
#pd.pivot_table(df,index=[u'对手',u'主客场'])

In [10]:
#手动填写错漏的数据
养殖场[养殖场['养殖场'] == '颍上2场']
S[350] = 50.67
l[350,0] = 500000

In [11]:
import time
start = time.clock()
a = ILP(l,T,S)
a.solve()
print('最低运输距离为：',a.opt_val)
print('最优供给关系为：','\n',a.opt_x)
elapsed = (time.clock() - start)
print("Time used:",elapsed)
#运算时间大概不到6秒， 结果是117*372的矩阵，0代表该饲料厂和养殖场没有供给关系，1代表由该饲料厂供给此养殖场

E:\jupyter\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


最低运输距离为： 9335872.0
最优供给关系为： 
      0    1    2    3    4    5    6    7    8    9    ...  362  363  364  \
0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3    0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
5    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
6    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
7    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
8    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
9    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
10   0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
11   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0

E:\jupyter\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys


In [66]:
#输出表格
S_name = pd.pivot_table(data,index=['养殖场'],columns = ['饲料厂'],values = '日需求量').index
T_name = pd.pivot_table(data,index=['养殖场'],columns = ['饲料厂'],values = '产能').columns
a.opt_x.columns = S_name
a.opt_x.index = T_name
a.opt_x.to_csv('result.csv')
result = a.opt_x

In [ ]:
# SQL部分，未完成
'''import pandas as pd
import pymysql
# sql 命令
sql_cmd1 = "SELECT * FROM table"
sql_cmd1 = "SELECT * FROM table"

con = pymysql.connect(host=localhost, user=username, password=password, database=dbname, charset='utf8', use_unicode=True)
#df1:养殖场及其需求，注意统一需求的时间，日需求或是月需求
df1 = pd.read_sql(sql_cmd1, con)
#df2 :饲料厂及其产能
df2 = pd.read_sql(sql_cmd2, con)
#data: 爬虫出来的距离矩阵
data = pd.read_csv('Data1.csv',encoding = 'gbk',sep= '\t',header = None).iloc[:,[1,5,9]]
#测试月份是2020年1月
data= pd.merge(data,data_1[data_1['月份']=='2020-01'],how = 'inner', on = '养殖场')
data =pd.merge(data,data_2,how = 'inner', on = '饲料厂')'''